This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '' #removed key for privacy

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [ ]:
# First, import the relevant modules
import requests

#### Task 1. Collect data from the Frankfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
specifics = '&start_date=2017-01-01&end_date=2017-12-31&collapse=daily'
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=' + API_KEY + specifics
r = requests.get(url)
json = r.json()['dataset_data']

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json.keys())
#for k in json.keys():
#    print(k + ': ', json[k])

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


#### Task 2. Convert the returned JSON object into a Python dictionary.

In [5]:
column_names = json['column_names']
dat = json['data']
print(column_names)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [6]:
data_dict = {k:[] for k in json['column_names']}
for row in dat:
    for i in range(len(data_dict.keys())):
        data_dict[column_names[i]].append(row[i])

#### Task 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [7]:
# isinstance was used to deal with missing values using base python, same list comprehension appears in following tasks as well
print("Highest opening price for Carl Zeiss Meditec was: ", max([x for x in data_dict['Open'] if isinstance(x, float) == True]))
print("Lowest opening price for Carl Zeiss Meditec was: ", min([x for x in data_dict['Open'] if isinstance(x, float) == True]))

Highest opening price for Carl Zeiss Meditec was:  53.11
Lowest opening price for Carl Zeiss Meditec was:  34.0


#### Task 4. What was the largest change in any one day (based on High and Low price)?

In [8]:
# zip was used to create tuples of High and Low price lists element-wise
# isinstance was used to check if the difference is a number to deal with missing values
# there is an assumption here that High is usually bigger than Low price
diff = [h-l for h,l in zip(data_dict['High'], data_dict['Low']) if isinstance(h-l, float) == True]
print("The largest price difference for Carl Zeiss Meditec within a single day was: ", round(max(diff), 2))

The largest price difference for Carl Zeiss Meditec within a single day was:  2.81


#### Task 5. What was the largest change between any two days (based on Closing Price)?

In [9]:
# assuming that the task means any two days in 2017, we get the max and min closing prices in all of 2017 data that is not missing
max_closing = max([x for x in data_dict['Close'] if isinstance(x, float) == True])
min_closing = min([x for x in data_dict['Close'] if isinstance(x, float) == True])
# we then find the index or location of max and min closing prices and in the print statement extract the corresponding dates using the indices
max_index = data_dict['Close'].index(max_closing)
min_index = data_dict['Close'].index(min_closing)
print("The largest change between closing prices of Carl Zeiss Meditec occured between", 
      data_dict['Date'][min_index], "(", data_dict['Close'][min_index], ") and ", data_dict['Date'][max_index], "(", data_dict['Close'][max_index], 
      ") \nfor the total difference of:", max_closing - min_closing)

The largest change between closing prices of Carl Zeiss Meditec occured between 2017-01-23 ( 34.06 ) and  2017-12-13 ( 53.09 ) 
for the total difference of: 19.03


In [10]:
# now assuming that the task refers to consecutive two days' difference in closing price
# we create an empty list and calculate consecutive difference using a for loop starting at index 1 position of closing price
change_cumulative = []
for i in range(1, len(data_dict['Close'])):
    change = data_dict['Close'][i] - data_dict['Close'][i-1]
    change_cumulative.append(change)
# we then find max and min
# note, it was possible to do this in a for loop, but since the data size was small, I did it after
max_change = max([x for x in change_cumulative if isinstance(x, float) == True])
min_change = min([x for x in change_cumulative if isinstance(x, float) == True])
# max change can be positive or negative, so we compare the absolute values of max and min and assign it to max_val
max_val = max_change if abs(max_change) > abs(min_change) else min_change
# we then find the index of max_val and add 1 because we skipped first element of closing price before to use it later to find the associated data and closing price
max_index = change_cumulative.index(max_val) + 1

print("The largest change between closing prices of Carl Zeiss Meditec from any two consecutive days occured on", data_dict['Date'][max_index], 
      "\nand the price of", data_dict['Close'][max_index], "on that date was changed from the previous date by", round(max_val, 2))

The largest change between closing prices of Carl Zeiss Meditec from any two consecutive days occured on 2017-08-08 
and the price of 44.37 on that date was changed from the previous date by 2.56


#### Task 6. What was the average daily trading volume during this year?

In [11]:
# using the same list comprehension as before, we remove missing values
clean_volume = [x for x in data_dict['Traded Volume'] if isinstance(x, float) == True]
# we then sum all elements in clean list and divide by number of elements to find the mean
avg_traded = sum(clean_volume) / len(clean_volume)
print("The average daily trading volume for Carl Zeiss Meditec in 2017 was", round(avg_traded, 2))

The average daily trading volume for Carl Zeiss Meditec in 2017 was 89124.34


#### Task 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
# we again clean the list to remove missing values
clean_volume = [x for x in data_dict['Traded Volume'] if isinstance(x, float) == True]
# we sort the list from smallest to largest
clean_volume.sort()
# print(clean_volume) if you want to verify it sorted correctly
# we calculate the number of elements in the list
n = len(clean_volume)
# if the number is even, we find the average of two middle elements
if n % 2 == 0:
    median_traded = (clean_volume[n//2] + clean_volume[n//2-1])/2
# else, we simply take the middle element as median
else:
    median_traded = clean_volume[n//2]
    
print("The median trading volume for Carl Zeiss Meditec in 2017 was", round(median_traded, 2))

The median trading volume for Carl Zeiss Meditec in 2017 was 76286.0
